# Dirty Data Cleaning: Autoencoder + Genetic Algorithm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from models.autoencoder import build_autoencoder
from utils.genetic_optimizer import run_genetic_algorithm

In [ ]:
df = pd.read_csv('../data/dirty_data.csv')
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['age'] = df['age'].mask((df['age'] < 10) | (df['age'] > 100))
df['age'].fillna(df['age'].median(), inplace=True)
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['income'] = df['income'].mask((df['income'] < 1000) | (df['income'] > 100000))
df['income'].fillna(df['income'].median(), inplace=True)

In [ ]:
X = df[['age', 'income']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
model = build_autoencoder(input_dim=2)
model.fit(X_scaled, X_scaled, epochs=10, batch_size=2, verbose=1)

In [ ]:
reconstructed = model.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=1)
df['anomaly_score'] = mse

In [ ]:
threshold = run_genetic_algorithm(df)
df_cleaned = df[df['anomaly_score'] < threshold]
df_cleaned.head()